# BYOL Demo

An end-to-end demonstration of BYOL in action, using STL10 as a toy dataset.

### Install dependencies

PyTorch and Torchvision are pre-installed in Colab instances, so no need to worry about those.

In [1]:
# Install dependencies.  Note that pytorch and torchvision are pre-installed 
# in standard Colab instances, so no need to worry about those.
!pip install -q kornia pytorch_lightning
!pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.1/628.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pytorch_lightning==1.9.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 kB 52.3 MB/s eta 0:00:00
  Attempting uninstall: pytorch_lightning
    Found existing installation: pytorch-lightning 2.0.1.post0
    Uninstalling pytorch-lightning-2.0.1.post0:
      Successfully uninstalled pytorch-lightning-2.0.1.post0


### Data Augmentations

In [3]:
import random
from typing import Callable, Tuple

from kornia import augmentation as aug
from kornia import filters
from kornia.geometry import transform as tf
import torch
from torch import nn, Tensor
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from os import cpu_count

from torch.utils.data import DataLoader
from torchvision.models import resnet18, resnet50


In [4]:
class RandomApply(nn.Module):
    def __init__(self, fn: Callable, p: float):
        super().__init__()
        self.fn = fn
        self.p = p

    def forward(self, x: Tensor) -> Tensor:
        return x if random.random() > self.p else self.fn(x)


def default_augmentation(image_size: Tuple[int, int] = (224, 224)) -> nn.Module:
    return nn.Sequential(
        tf.Resize(size=image_size),
        RandomApply(aug.ColorJitter(0.8, 0.8, 0.8, 0.2), p=0.8),
        aug.RandomGrayscale(p=0.2),
        aug.RandomHorizontalFlip(),
        RandomApply(filters.GaussianBlur2d((3, 3), (1.5, 1.5)), p=0.1),
        aug.RandomResizedCrop(size=image_size),
        aug.Normalize(
            mean=torch.tensor([0.485, 0.456, 0.406]),
            std=torch.tensor([0.229, 0.224, 0.225]),
        ),
    )

### Encoder Wrapper

Enables us to work with *any* model, not just ResNet18 which I chose for this demo.

In [5]:
from typing import Union


def mlp(dim: int, projection_size: int = 256, hidden_size: int = 4096) -> nn.Module:
    return nn.Sequential(
        nn.Linear(dim, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size),
    )


class EncoderWrapper(nn.Module):
    def __init__(
        self,
        model: nn.Module,
        projection_size: int = 256,
        hidden_size: int = 4096,
        layer: Union[str, int] = -2,
    ):
        super().__init__()
        self.model = model
        self.projection_size = projection_size
        self.hidden_size = hidden_size
        self.layer = layer

        self._projector = None
        self._projector_dim = None
        self._encoded = torch.empty(0)
        self._register_hook()

    @property
    def projector(self):
        if self._projector is None:
            self._projector = mlp(
                self._projector_dim, self.projection_size, self.hidden_size
            )
        return self._projector

    def _hook(self, _, __, output):
        output = output.flatten(start_dim=1)
        if self._projector_dim is None:
            self._projector_dim = output.shape[-1]
        self._encoded = self.projector(output)

    def _register_hook(self):
        if isinstance(self.layer, str):
            layer = dict([*self.model.named_modules()])[self.layer]
        else:
            layer = list(self.model.children())[self.layer]

        layer.register_forward_hook(self._hook)

    def forward(self, x: Tensor) -> Tensor:
        _ = self.model(x)
        return self._encoded

### BYOL and Training Code

Encapsulate BYOL into a single module.  I use PyTorch Lightning here, because it makes training very easy.  This code also works for multi-GPU or TPU training, and experiments are logged automatically.

In [18]:
from copy import deepcopy
from itertools import chain
from typing import Dict, List

import pytorch_lightning as pl
from torch import optim
import torch.nn.functional as f


def normalized_mse(x: Tensor, y: Tensor) -> Tensor:
    x = f.normalize(x, dim=-1)
    y = f.normalize(y, dim=-1)
    return 2 - 2 * (x * y).sum(dim=-1)


class BYOL(pl.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        image_size: Tuple[int, int] = (128, 128),
        hidden_layer: Union[str, int] = -2,
        projection_size: int = 256,
        hidden_size: int = 4096,
        augment_fn: Callable = None,
        beta: float = 0.999,
        **hparams,
    ):
        super().__init__()
        self.augment = default_augmentation(image_size) if augment_fn is None else augment_fn
        self.beta = beta
        self.encoder = EncoderWrapper(
            model, projection_size, hidden_size, layer=hidden_layer
        )
        self.predictor = nn.Linear(projection_size, projection_size, hidden_size)
        # self.hparams = hparams
        self._target = None

        self.encoder(torch.zeros(2, 3, *image_size))

    def forward(self, x: Tensor) -> Tensor:
        return self.predictor(self.encoder(x))

    @property
    def target(self):
        if self._target is None:
            self._target = deepcopy(self.encoder)
        return self._target

    def update_target(self):
        for p, pt in zip(self.encoder.parameters(), self.target.parameters()):
            pt.data = self.beta * pt.data + (1 - self.beta) * p.data

    # --- Methods required for PyTorch Lightning only! ---

    def configure_optimizers(self):
        optimizer = getattr(optim, self.hparams.get("optimizer", "Adam"))
        lr = self.hparams.get("lr", 1e-4)
        weight_decay = self.hparams.get("weight_decay", 1e-6)
        return optimizer(self.parameters(), lr=lr, weight_decay=weight_decay)

    def training_step(self, batch, *_) -> Dict[str, Union[Tensor, Dict]]:
        x = batch[0]
        with torch.no_grad():
            x1, x2 = self.augment(x), self.augment(x)

        pred1, pred2 = self.forward(x1), self.forward(x2)
        with torch.no_grad():
            targ1, targ2 = self.target(x1), self.target(x2)
        loss = torch.mean(normalized_mse(pred1, targ2) + normalized_mse(pred2, targ1))

        self.log("train_loss", loss.item())
        self.update_target()

        return {"loss": loss}

    @torch.no_grad()
    def validation_step(self, batch, *_) -> Dict[str, Union[Tensor, Dict]]:
        x = batch[0]
        x1, x2 = self.augment(x), self.augment(x)
        pred1, pred2 = self.forward(x1), self.forward(x2)
        targ1, targ2 = self.target(x1), self.target(x2)
        loss = torch.mean(normalized_mse(pred1, targ2) + normalized_mse(pred2, targ1))

        return {"loss": loss}

    @torch.no_grad()
    def validation_epoch_end(self, outputs: List[Dict]) -> Dict:
        val_loss = sum(x["loss"] for x in outputs) / len(outputs)
        self.log("val_loss", val_loss.item())

### Supervised Training Module



In [19]:
class SupervisedLightningModule(pl.LightningModule):
    def __init__(self, model: nn.Module, **hparams):
        super().__init__()
        self.model = model
        self.save_hyperparameters()

    def forward(self, x: Tensor) -> Tensor:
        return self.model(x)

    def configure_optimizers(self):
        optimizer = getattr(optim, self.hparams.get("optimizer", "Adam"))
        lr = self.hparams.get("lr", 1e-4)
        weight_decay = self.hparams.get("weight_decay", 1e-6)
        return optimizer(self.parameters(), lr=lr, weight_decay=weight_decay)

    def training_step(self, batch, *_) -> Dict[str, Union[Tensor, Dict]]:
        x, y = batch
        loss = f.cross_entropy(self.forward(x), y)
        self.log("train_loss", loss.item(), prog_bar=True, on_step=False, on_epoch=True)
        return {"loss": loss}

    @torch.no_grad()
    def validation_step(self, batch, *_) -> Dict[str, Union[Tensor, Dict]]:
        x, y = batch
        loss = f.cross_entropy(self.forward(x), y)
        self.log("val_loss", loss.item(), prog_bar=True, on_step=False, on_epoch=True) 
        return {"loss": loss}

    @torch.no_grad()
    def validation_epoch_end(self, outputs: List[Dict]) -> Dict:
        val_loss = sum(x["loss"] for x in outputs) / len(outputs)
        self.log("val_loss", val_loss.item())

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!unrar x "/content/drive/MyDrive/Capstone dataset/FGNET73.rar"


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/drive/MyDrive/Capstone dataset/FGNET73.rar

Creating    FGNET73                                                   OK
Creating    FGNET73/test                                              OK
Creating    FGNET73/test/001                                          OK
Extracting  FGNET73/test/001/001A28.JPG                                    0%  OK 
Extracting  FGNET73/test/001/001A33.JPG                                    0%  OK 
Extracting  FGNET73/test/001/001A43b.JPG                                   0%  OK 
Creating    FGNET73/test/002                                          OK
Extracting  FGNET73/test/002/002A26.JPG                                    0%  OK 
Extracting  FGNET73/test/002/002A36.JPG                                    0%  OK 
Extracting  FGNET73/test/002/002A38.JPG                                    0%  OK 
Creating   

### Supervised Training without BYOL

Run through supervised training, and measure the accuracy.  Performance should be pretty good already. 

In [10]:
from torchvision import datasets, models, transforms

input_path = "/content/FGNET73"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),

    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + '/train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + '/val', data_transforms['validation']),
    'test': 
    datasets.ImageFolder(input_path + '/test', data_transforms['test'])
}


train_dataloader = torch.utils.data.DataLoader(image_datasets['train'],
                                 batch_size=32,
                                 shuffle=True)
val_dataloader = torch.utils.data.DataLoader(image_datasets['validation'],
                                 batch_size=32,
                                 shuffle=False)
test_dataloader = torch.utils.data.DataLoader(image_datasets['test'],
                                 batch_size=32,
                                 shuffle=False)

class_names = image_datasets['train'].classes

In [11]:
#load pretrain
model = resnet18(pretrained=True)
num_ftrs = model.fc.in_features  
model.fc = nn.Sequential(
               nn.Linear(num_ftrs, 256),
               nn.ReLU(inplace=True),
               nn.Dropout(0.2),
               nn.Linear(256, len(class_names)))



/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 338MB/s]


In [12]:
# from torchinfo import summary
# summary(model, input_size=(8, 3, 224, 224))

In [13]:
checkpoint_callback = ModelCheckpoint(
                                      dirpath='/content/checkpoint_model/',
                                      filename='sample-fgnet-{epoch:02d}-{val_loss:.2f}',
                                      save_top_k = 1,
                                      monitor='val_loss',
                                      mode='min'
                                      )

In [14]:
early_stop_callback = EarlyStopping(
                                    monitor="val_loss",
                                    mode="min",
                                    patience=10)

In [15]:
# from pytorch_lightning.loggers import TensorBoardLogger
# logger = TensorBoardLogger("tb_logs", name="my_model")

In [20]:
supervised = SupervisedLightningModule(model)
trainer = pl.Trainer(
                     max_epochs=30,
                     gpus=-1,
                     callbacks=[checkpoint_callback,early_stop_callback],
                     enable_model_summary=None)

trainer.fit(supervised, train_dataloader, val_dataloader)

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
def accuracy(pred: Tensor, labels: Tensor) -> float:
    return (pred.argmax(dim=-1) == labels).float().mean().item()

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/MINH/FGNET_r18_baseline_73.pth')

In [ ]:
import shutil
import glob
t = glob.glob('/content/checkpoint_model/*')
print(t)
name = t[0].split("/")[-1]
shutil.copy(t[0], '/content/drive/MyDrive/MINH/7-3/'+name)

In [ ]:
test2 = SupervisedLightningModule.load_from_checkpoint('/content/drive/MyDrive/MINH/7-3/sample-fgnet-epoch=39-val_loss=1.93.ckpt')
test2.eval()


test2.cuda()
acc = sum([accuracy(test2(x.cuda()), y.cuda()) for x, y in test_dataloader]) / len(test_dataloader)
print(f"Accuracy: {acc:.3f}")
#clear cache
torch.cuda.empty_cache()

Accuracy: 0.513


### Self-Supervised Training with BYOL

Perform our self-supervised training.  This is the most computationally intensive part of the script.  It takes roughly 45 minutes on a standard Colab instance with a K80 GPU.

In [ ]:
checkpoint_callback = ModelCheckpoint(
                                      dirpath='/content/checkpoint_BYOL/',
                                      filename='BYOL-fgnet-{epoch:02d}-{val_loss:.2f}',
                                      save_top_k = 1,
                                      monitor='val_loss',
                                      mode='min'
                                      )

In [ ]:
model_SSL = resnet18(pretrained = True)
num_ftrs = model_SSL.fc.in_features
model_SSL.fc = nn.Sequential(
               nn.Linear(num_ftrs, 256),
               nn.ReLU(inplace=True),
               nn.Dropout(0.2),
               nn.Linear(256, len(class_names)))


byol = BYOL(model_SSL, image_size=(96, 96))
trainer = pl.Trainer(
    max_epochs=50, 
    gpus=-1,
    accumulate_grad_batches=2048 // 128,
    callbacks = [checkpoint_callback],
)

trainer.fit(byol, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /content/checkpoint_BYOL exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type           | Params
---------------------------------------------
0 | augment   | Sequential     | 0     
1 | encoder   | EncoderWrapper | 14.5 M
2 | predictor | Linear         | 65.8 K
---------------------------------------------
14.6 M    Trainable params
0         Non

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


### Supervised Training Again

Extract the state dictionary from BYOL, and load it into our ResNet18 model before starting training.  Then run supervised training, and watch the accuracy improve from last time!

In [ ]:
state_dict = model_SSL.state_dict()
model_improved = resnet18()
num_ftrs = model_improved.fc.in_features
model_improved.fc = nn.Sequential(
               nn.Linear(num_ftrs, 256),
               nn.ReLU(inplace=True),
               nn.Dropout(0.1),
               nn.Linear(256, len(class_names)))
model_improved.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
checkpoint_callback = ModelCheckpoint(
                                      dirpath='/content/checkpoint_improved/',
                                      filename='improved-fgnet-{epoch:02d}-{val_loss:.2f}',
                                      save_top_k = 1,
                                      monitor='val_loss',
                                      mode='min'
                                      )

In [ ]:
supervised = SupervisedLightningModule(model_improved)
trainer = pl.Trainer(
    max_epochs=40,
    gpus=-1,
    callbacks = [checkpoint_callback, early_stop_callback],
    enable_model_summary=None,
)

trainer.fit(supervised, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


In [ ]:
torch.save(model_improved.state_dict(), '/content/drive/MyDrive/MINH/FGNET_r18_improved_73_1.pth')

In [ ]:
import shutil
import glob
t = glob.glob('/content/checkpoint_model/*')
print(t)
name = t[0].split("/")[-1]
shutil.copy(t[0], '/content/drive/MyDrive/MINH/7-3/'+name)

In [ ]:
test2 = SupervisedLightningModule.load_from_checkpoint('/content/drive/MyDrive/MINH/7-3/improved-fgnet-epoch=22-val_loss=1.47.ckpt')
test2.eval()


test2.cuda()
acc = sum([accuracy(test2(x.cuda()), y.cuda()) for x, y in test_dataloader]) / len(test_dataloader)
print(f"Accuracy: {acc:.3f}")
#clear cache
torch.cuda.empty_cache()

Accuracy: 0.519
